In [1]:
# AutoGluon Version:  0.4.0

In [19]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.tabular import version
version.__version__

'0.4.0'

# Training AutoGluon Model

In [3]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
# subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
# train_data = train_data.sample(n=subsample_size, random_state=0)

In [4]:
label = 'class'
save_path = 'app/ag_models'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(
    train_data, 
    presets='optimize_for_deployment',
    excluded_model_types=['NN', 'KNN'], # no mxnet in the container; knn takes lots of space
)

Presets specified: ['optimize_for_deployment']
Beginning AutoGluon training ...
AutoGluon will save models to "app/ag_models/"
AutoGluon Version:  0.4.0
Python Version:     3.8.12
Operating System:   Darwin
Train Data Rows:    39073
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generat

In [5]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


# Checking If Prediction Works

In [6]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


Evaluation: accuracy on test data: 0.8761388064284983
Evaluations on test data:
{
    "accuracy": 0.8761388064284983,
    "balanced_accuracy": 0.799329968158955,
    "mcc": 0.6408943069377524,
    "f1": 0.7144879660217084,
    "precision": 0.7885416666666667,
    "recall": 0.6531492666091459
}


Predictions:  
 0        <=50K
1        <=50K
2         >50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object


# Preparing a Message for API Gateway call

In [7]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


In [8]:
import pandas as pd

In [9]:
test_data_nolab.head().to_json()

'{"age":{"0":31,"1":17,"2":47,"3":21,"4":17},"workclass":{"0":" Private","1":" Self-emp-not-inc","2":" Private","3":" Private","4":" Private"},"fnlwgt":{"0":169085,"1":226203,"2":54260,"3":176262,"4":241185},"education":{"0":" 11th","1":" 12th","2":" Assoc-voc","3":" Some-college","4":" 12th"},"education-num":{"0":7,"1":8,"2":11,"3":10,"4":8},"marital-status":{"0":" Married-civ-spouse","1":" Never-married","2":" Married-civ-spouse","3":" Never-married","4":" Never-married"},"occupation":{"0":" Sales","1":" Sales","2":" Exec-managerial","3":" Exec-managerial","4":" Prof-specialty"},"relationship":{"0":" Wife","1":" Own-child","2":" Husband","3":" Own-child","4":" Own-child"},"race":{"0":" White","1":" White","2":" White","3":" White","4":" White"},"sex":{"0":" Female","1":" Male","2":" Male","3":" Female","4":" Male"},"capital-gain":{"0":0,"1":0,"2":0,"3":0,"4":0},"capital-loss":{"0":0,"1":0,"2":1887,"3":0,"4":0},"hours-per-week":{"0":20,"1":45,"2":60,"3":30,"4":20},"native-country":{"0